# Proces weryfikacji nauczonej sieci Cz. 2

In [1]:
from Veryfication import VoiceSamples

Type model path: C:/Users/AXDition/Documents/GitHub/NN_VoiceRecognition_Models


Wynik rozpoznawania nagrań Krystiana!
Krystian: 39  Nicia: 1


Wynik rozpoznawania nagrań Nici!
Krystian: 1  Nicia: 23


Przykładowe wyjście nauczonej sieci dla próbki Krystiana:
tensor([[ 1.0827, -0.1323]], grad_fn=<AddmmBackward>)


Przykładowe wyjście nauczonej sieci dla próbki Nici:
tensor([[0.1956, 0.5329]], grad_fn=<AddmmBackward>)


## Wnioski 
- Sieć nauczyła się prawidłowo
- Skuteczność sieci można wstępnie oszacować na ponad 90%
- Sieć jest nieoptymalna (model zajmuje przeszło 600MB, za duża ilość neuronów)

Niżej zaprezentowane funkcje są wyrywkami zawartymi w pliku Veryfikacja.py wywołanym powyżej

In [ ]:
# loading test samples of Krystians Voice
vsKrystianTest = VoiceSamples("vsKrystianTest", samples_path="database/Test/" , Automatic=True)


K = 0 # Recognizes as Krystian
N = 0 # Recognizes as Nicia

# Recognition loop

for k in range(len(vsKrystianTest)):  
    
    vs = vsKrystianTest[k]
    
    input = vs.view(-1,1,256,256)
    output = net(input)
    v,i = output[0].max(0)
    if int(i) == 0:
        K += 1
    else:
        N += 1
   
print("\n")
print("Wynik rozpoznawania nagrań Krystiana!")
print("Krystian:",K," Nicia:",N)

In [ ]:
# example outputs 

input = vsKrystianTest[5].view(-1,1,256,256)
output = net(input)

print("\n")
print("Przykładowe wyjście nauczonej sieci dla próbki Krystiana:")
print(output)

input = vsNiciaTest[5].view(-1,1,256,256)
output = net(input)

print("\n")
print("Przykładowe wyjście nauczonej sieci dla próbki Nici:")
print(output)